In [47]:
from matplotlib import pyplot as plt
import numpy as np
import math
import meep as mp
import cmath

In [53]:
pixel_size = 5
period = 0.6/pixel_size #0.12
norm_wav_len = 1.18 / period #9.833
freq = 1 / norm_wav_len #0.1017
init_arr = np.random.randint(2, size = pixel_size)
h = 0.75/period # 6.25
dpml = 0.5*norm_wav_len #4.9165
sx, sy, sz = pixel_size, 1, 2*h+2*dpml #5, 1, 22.133/11.066

b_m, c_m = 1.4, 3.54
resolution = 20
cell_size = mp.Vector3(sx,sy,sz)
fcen = freq
df = 0.2
nfreq = 200

def pw_amp(k, x0):
    def _pw_amp(x):
        return cmath.exp(1j * 2 * math.pi * k.dot(x + x0))
    return _pw_amp

pixel_arr = np.zeros(pixel_size)

for i in range(pixel_size):
#     if init_arr[i] == 1:
#         pixel_arr[i] = c_m
    pixel_arr[i] = c_m
print(pixel_arr)

[3.54 3.54 3.54 3.54 3.54]


In [64]:
geometry = []
for i in range(pixel_size):
    if pixel_arr[i] == 0:
        continue
    geometry.append(mp.Block(center = mp.Vector3(-sx/2+0.5+i,0,0),
                             size = mp.Vector3(1,1,h),
                             material = mp.Medium(index=pixel_arr[i])))
    
#pml_layers = [mp.PML(dpml)]

pml_layers = [mp.PML(thickness=dpml, direction = mp.Z, side=mp.High),
              mp.Absorber(thickness=dpml,direction = mp.Z, side=mp.Low)]

deta_sourceZ_forw = -(sz/2 - dpml - 1.5)
deta_sourceZ_back = -deta_sourceZ_forw

theta_forw = math.radians(0)
k_forw = mp.Vector3(math.sin(theta_forw),0,math.cos(theta_forw)).scale(fcen)
theta_back = math.radians(30)
k_back = mp.Vector3(math.sin(theta_back),0,math.cos(theta_back)).scale(fcen)

sources_forw = [mp.Source(src = mp.GaussianSource(frequency = freq), 
                          component = mp.Ey,
                          center = mp.Vector3(0,0,deta_sourceZ_forw), 
                          size = mp.Vector3(sx,sy,0),
                          amp_func = pw_amp(k_forw,mp.Vector3(0,0,deta_sourceZ_forw)))]

sources_back = [mp.Source(src = mp.GaussianSource(frequency = freq), 
                          component = mp.Ey,
                          center = mp.Vector3(0,0,deta_sourceZ_back), 
                          size = mp.Vector3(sx,sy,0),
                          amp_func = pw_amp(k_back,mp.Vector3(0,0,deta_sourceZ_back)))]

sim_forw = mp.Simulation(cell_size = cell_size, 
                         boundary_layers = pml_layers,
                         geometry = geometry, 
                         sources = sources_forw, 
                         resolution = resolution, 
                         #force_complex_fields = True,
                         k_point = k_forw)

sim_back = mp.Simulation(cell_size = cell_size, 
                         boundary_layers = pml_layers,
                         geometry = geometry, 
                         sources = sources_back, 
                         resolution = resolution, 
                         #force_complex_fields = True,
                         k_point = k_back)

In [59]:
# sim_forw.init_sim()
# eps_data = sim_forw.get_epsilon()
# from mayavi import mlab
# s = mlab.contour3d(eps_data, colormap="YlGnBu")
# mlab.show()

In [65]:
#sim.use_output_directory()
sim_forw.run(until = 30)#30
sim_back.run(until = 30)#30


ez_data_forw = sim_forw.get_array(center=mp.Vector3(0,0,0), size=mp.Vector3(sx,sy,h), component=mp.Ey)
ez_data_back = sim_back.get_array(center=mp.Vector3(0,0,0), size=mp.Vector3(sx,sy,h), component=mp.Ey)

-----------
Initializing structure...
time for choose_chunkdivision = 0.000120878 s
Working in 3D dimensions.
Computational cell is 5 x 1 x 22.35 with resolution 20
     block, center = (-2,0,0)
          size (1,1,6.25)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (12.5316,12.5316,12.5316)
     block, center = (-1,0,0)
          size (1,1,6.25)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (12.5316,12.5316,12.5316)
     block, center = (0,0,0)
          size (1,1,6.25)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (12.5316,12.5316,12.5316)
     block, center = (1,0,0)
          size (1,1,6.25)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (12.5316,12.5316,12.5316)
     block, center = (2,0,0)
          size (1,1,6.25)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (12.

In [66]:
[m_x,m_y,m_z]=(ez_data_forw.shape)

mul_array = np.multiply(ez_data_forw, ez_data_back)
summed = np.sum(mul_array, axis = 2)
real_sum = summed.real

real_sum_reduced=np.zeros((len(range(0,m_x,resolution)), len(range(0,m_y,resolution))))
print(real_sum_reduced.shape)
print(real_sum_reduced)
for i in range(0,m_x,resolution):
    for j in range(0,m_y,resolution):
        real_sum_reduced[int(i/resolution),int(j/resolution)]=sum(sum(real_sum[i:i+resolution,j:j+resolution]))

#how to get the y axis value of mul_array.shape
[x_size, y_size] = (real_sum_reduced.shape)
permitivity_change = real_sum_reduced[:,int(y_size/2)]

print(permitivity_change.shape)
print(permitivity_change)
C=0.015
Updated_permitivity = pixel_arr + permitivity_change*C

for i in range(0,x_size):
    if Updated_permitivity[i]<n_PDMS**2:
        Updated_permitivity[i]=n_PDMS**2

(6, 2)
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
(6,)
[0. 0. 0. 0. 0. 0.]


ValueError: operands could not be broadcast together with shapes (5,) (6,) 